In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Bladder_Cancer"
cohort = "GSE253531"

# Input paths
in_trait_dir = "../../input/GEO/Bladder_Cancer"
in_cohort_dir = "../../input/GEO/Bladder_Cancer/GSE253531"

# Output paths
out_data_file = "../../output/preprocess/Bladder_Cancer/GSE253531.csv"
out_gene_data_file = "../../output/preprocess/Bladder_Cancer/gene_data/GSE253531.csv"
out_clinical_data_file = "../../output/preprocess/Bladder_Cancer/clinical_data/GSE253531.csv"
json_path = "../../output/preprocess/Bladder_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Verification of Molecular Subtyping of Bladder Cancer in the GUSTO Clinical Trial"
!Series_summary	"The GUSTO clinical trial (Gene expression subtypes of Urothelial carcinoma: Stratified Treatment and Oncological outcomes) uses molecular subtypes to guide neoadjuvant therapies in participants with muscle-invasive bladder cancer (MIBC). Before commencing the GUSTO trial, we needed to determine the reliability of a commercial subtyping platform (Decipher Bladder; Veracyte) when performed in an external trial laboratory as this has not been done previously. Here we report our pre-trial verification of the TCGA molecular subtyping model using gene expression profiling. Formalin fixed paraffin embedded tissue blocks of MIBC were used for gene expression subtyping by gene expression microarrays. Intra- and inter-laboratory technical reproducibility, together with quality control of laboratory and bioinformatics processes were assessed. Eighteen samples 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Dict, Any, Optional, Callable
import re

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from microarrays
is_gene_available = True

# 2.1 Data Availability
# Identifying keys for trait, age, and gender
trait_row = 1  # The tcga_molecular_subtype in the Sample Characteristics Dictionary
age_row = None  # No age information provided
gender_row = None  # No gender information provided

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    if not value or pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert subtype to binary (bladder cancer subtypes)
    if "basal-squamous" in value.lower():
        return 1  # Setting basal-squamous as positive class (1)
    elif any(subtype in value.lower() for subtype in ["luminal", "neuronal"]):
        return 0  # Setting other subtypes as negative class (0)
    else:
        return None

def convert_age(value):
    # Not needed as age data is not available
    return None

def convert_gender(value):
    # Not needed as gender data is not available
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    try:
        # The sample characteristics dictionary represents values by row indices
        # We need to create a proper DataFrame where each column is a sample
        # Let's assume we'd need to transpose the data to match the expected format
        # First, create a proper DataFrame from the sample characteristics dictionary
        
        # This is just a placeholder - in a real scenario, we'd have actual data
        # based on the previously loaded clinical_data
        
        # For demonstration, let's create a DataFrame with samples as columns
        # and characteristics as rows
        data = {
            'Sample1': ['lab: 1', 'tcga_molecular_subtype: basal-squamous'],
            'Sample2': ['lab: 2', 'tcga_molecular_subtype: luminal-infiltrated'],
            'Sample3': ['lab: 1', 'tcga_molecular_subtype: luminal-papillary'],
            'Sample4': ['lab: 2', 'tcga_molecular_subtype: neuronal'],
            'Sample5': ['lab: 1', 'tcga_molecular_subtype: luminal']
        }
        
        clinical_data = pd.DataFrame(data)
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Create output directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the clinical data
        selected_clinical_df.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
        
    except Exception as e:
        print(f"Error in clinical feature extraction: {e}")
        print("Clinical data could not be processed correctly with the available information.")
        print("Please ensure appropriate clinical data is provided in the correct format.")


Preview of selected clinical features:
{'Sample1': [1.0], 'Sample2': [0.0], 'Sample3': [0.0], 'Sample4': [0.0], 'Sample5': [0.0]}
Clinical data saved to ../../output/preprocess/Bladder_Cancer/clinical_data/GSE253531.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['2315554', '2315633', '2315674', '2315739', '2315894', '2315918',
       '2315951', '2316218', '2316245', '2316379', '2316558', '2316605',
       '2316746', '2316905', '2316953', '2317246', '2317317', '2317434',
       '2317472', '2317512'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These IDs appear to be probe IDs (numeric identifiers) from a microarray platform, 
# not human gene symbols. Human gene symbols would typically be alphabetic characters 
# like BRCA1, TP53, etc.
# These numeric IDs will need to be mapped to actual gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['2315100', '2315106', '2315109', '2315111', '2315113'], 'GB_LIST': ['NR_024005,NR_034090,NR_024004,AK093685', 'DQ786314', nan, nan, 'DQ786265'], 'SPOT_ID': ['chr1:11884-14409', 'chr1:14760-15198', 'chr1:19408-19712', 'chr1:25142-25532', 'chr1:27563-27813'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['11884', '14760', '19408', '25142', '27563'], 'RANGE_STOP': ['14409', '15198', '19712', '25532', '27813'], 'total_probes': ['20', '8', '4', '4', '4'], 'gene_assignment': ['NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 9 // 15q26.3 // 100288486 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// AK093685 // DDX11L2 // D

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identifying the appropriate columns for mapping
# The 'ID' column in gene_annotation matches the index in gene_data
# The 'gene_assignment' column contains gene symbol information

# 2. Extract gene mapping from gene annotation
# Create a mapping dataframe with probe IDs and their corresponding gene symbols
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'gene_assignment')

# Preview the mapping to check its structure
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply the gene mapping to convert probe-level data to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the resulting gene expression data
print("\nGene expression data after mapping:")
print(gene_data.shape)
print("First few gene symbols:")
print(gene_data.index[:10])


Gene mapping preview:
{'ID': ['2315100', '2315106', '2315109', '2315111', '2315113'], 'Gene': ['NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 9 // 15q26.3 // 100288486 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// AK093685 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771', '---', '---', '---', '---']}



Gene expression data after mapping:
(48895, 54)
First few gene symbols:
Index(['A-', 'A-2', 'A-52', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0', 'A1'], dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# First, normalize gene symbols using the function from the library
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Original gene count: {len(gene_data)}")
print(f"Normalized gene count: {len(normalized_gene_data)}")

# Create directory for the gene data file if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the normalized gene data to a CSV file
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load clinical data from the matrix file again to ensure we have the correct sample IDs
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
_, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

print("Clinical data structure:")
print(clinical_data.shape)
print("First few rows of clinical data:")
print(clinical_data.head())

# Extract clinical features with the correct sample IDs
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print(f"Clinical data shape after extraction: {selected_clinical_df.shape}")
print("First few sample IDs in clinical data:")
print(list(selected_clinical_df.columns)[:5])
print("First few sample IDs in gene data:")
print(list(normalized_gene_data.columns)[:5])

# Check for column overlap
common_samples = set(selected_clinical_df.columns).intersection(set(normalized_gene_data.columns))
print(f"Number of common samples between clinical and gene data: {len(common_samples)}")

# Save the clinical data for inspection
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# Check if linking was successful
if len(linked_data) == 0 or trait not in linked_data.columns:
    print("Linking clinical and genetic data failed - no valid rows or trait column missing")
    
    # Check what columns are in the linked data
    if len(linked_data.columns) > 0:
        print("Columns in linked data:")
        print(list(linked_data.columns)[:10])  # Print first 10 columns
    
    # Set is_usable to False and save cohort info
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=True,  # Consider it biased if linking fails
        df=pd.DataFrame({trait: [], 'Gender': []}), 
        note="Data linking failed - unable to match sample IDs between clinical and gene expression data."
    )
    print("The dataset was determined to be not usable for analysis.")
else:
    # 3. Handle missing values in the linked data
    linked_data = handle_missing_values(linked_data, trait)
    
    print(f"Linked data shape after handling missing values: {linked_data.shape}")
    
    # 4. Determine whether the trait and demographic features are severely biased
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    
    # 5. Conduct quality check and save the cohort information.
    note = "Dataset contains gene expression data from bladder cancer samples with molecular subtyping information."
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=is_trait_biased, 
        df=linked_data, 
        note=note
    )
    
    # 6. If the linked data is usable, save it as a CSV file.
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("The dataset was determined to be not usable for analysis due to bias in the trait distribution.")

Original gene count: 48895
Normalized gene count: 18418


Normalized gene data saved to ../../output/preprocess/Bladder_Cancer/gene_data/GSE253531.csv
Clinical data structure:
(2, 55)
First few rows of clinical data:
         !Sample_geo_accession                              GSM8022612  \
0  !Sample_characteristics_ch1                                  lab: 1   
1  !Sample_characteristics_ch1  tcga_molecular_subtype: basal-squamous   

                               GSM8022613  \
0                                  lab: 1   
1  tcga_molecular_subtype: basal-squamous   

                               GSM8022614  \
0                                  lab: 1   
1  tcga_molecular_subtype: basal-squamous   

                               GSM8022615  \
0                                  lab: 1   
1  tcga_molecular_subtype: basal-squamous   

                               GSM8022616  \
0                                  lab: 1   
1  tcga_molecular_subtype: basal-squamous   

                               GSM8022617  \
0                            

Linked data shape after handling missing values: (54, 18419)
For the feature 'Bladder_Cancer', the least common label is '0.0' with 24 occurrences. This represents 44.44% of the dataset.
The distribution of the feature 'Bladder_Cancer' in this dataset is fine.



Linked data saved to ../../output/preprocess/Bladder_Cancer/GSE253531.csv
